In [1]:
import requests 
from bs4 import BeautifulSoup 
import pandas as pd

#Storing URLS 
Lhr_URL = "http://lhr.nu.edu.pk/faculty/"

##### WEB SCRAPING FOR LHR WEBSITE #####
req1 = requests.get(Lhr_URL)
soup1 = BeautifulSoup(req1.content, 'html.parser')

#Declaring Arrays for Storing Data
Id = []
Name = []
Designation = []
HEC_Status = []
Highest_Education = []
Email = []
Department = []
Extension = []
ImageURL = []
Profile_URL = []

#Dividing With Respect With Containers
containers = soup1.findAll('div', class_='container', id=['fsc','ee','cv','fsm','ss'])

for i in range(len(containers)):
    Cont=containers[i]
    
    #Finding Each Faculty Card in A Container
    fac1 = Cont.findAll('div', class_='facultyCard')
    
    Dep=Cont.find('h1',class_="mb-2 mt-3").text
    
    for j in range(len(fac1)):
        Faculty = fac1[j]
            
        #Id
        try:
            Id.append(int(Faculty.find('a',class_='faculty-link')['href'].split('/')[-1]))
        except:
            Id.append(-1)

        #Name
        Name.append(Faculty.find('h5', class_='text-center').text)

        #Designation
        Temp = [item.strip() for item in Faculty.find('p', class_='small').text.strip().split('\n') if item.strip()]
        Designation.append(Temp[0])

        #HEC Status
        if len(Temp)>1:
            HEC_Status.append(True)
        else:
            HEC_Status.append(False)

        #Email
        Email.append(Faculty.find('p', class_='mb-0').text)

        #Department
        Department.append(Dep)
        
        #Image URL
        ImageURL.append('https://lhr.nu.edu.pk'+Faculty.find('img', class_='card-img-top')['src'])

        #Profile URL
        Profile_URL.append('https://lhr.nu.edu.pk'+Faculty.find('a',class_='faculty-link')['href'])
        
for i in range(len(Profile_URL)):
    prof = Profile_URL[i]

    req2 = requests.get(prof)
    soup2 = BeautifulSoup(req2.content, 'html.parser')
    
    #Extension
    tmp = soup2.find('span', class_='small').text.split(':')[-1].strip()
    if tmp=='None':
        tmp=-1
        Extension.append(int(tmp))
    else:
        Extension.append(int(tmp))
       
    #Highest Education
    Highest_Education.append(next((li.text.strip() for li in soup2.select('.text-justify li')), "No Education Record"))

    
# Create a dictionary from the arrays
data = {
    'Name': Name,
    'Designation': Designation,
    'Email': Email,
    'Extension':Extension,
    'ImageURL':ImageURL,
    'ProfileURL':Profile_URL,
    'HECstatus':HEC_Status,
    'Highest_Education':Highest_Education,
    'Extension':Extension,
    'Department':Department
}

# Convert the dictionary to a DataFrame
df1 = pd.DataFrame(data)

In [23]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

Khi_URL = "https://khi.nu.edu.pk/"

#Declaring Arrays for Storing Data
Id = []
Name = []
Designation = []
HEC_Status = []
Highest_Education = []
Email = []
Department = []
Extension = []
ImageURL = []
Profile_URL = []

req1 = requests.get(Khi_URL)
soup1 = BeautifulSoup(req1.content, 'html.parser')

sub = soup1.find_all('ul', class_='sub-menu')[1]

List_URL = []

# Traverse the items in the second submenu and extract their URLs
for i in sub.find_all('li'):
    a_tag = i.find('a')
    if a_tag:
        href = a_tag['href']
        if href != '#':
            List_URL.append(href)

for i in range(len(List_URL)):
    temp = List_URL[i]
    
    req2 = requests.get(temp)
    soup2 = BeautifulSoup(req2.content, 'html.parser')
    fac_Card = soup2.findAll('div', class_='gdlr-core-personnel-list-column')
    
    
    for j in range(len(fac_Card)):
        Faculty=fac_Card[j]
    
        #Name
        Name.append(Faculty.find('h3', class_='gdlr-core-personnel-list-title').find('a').text.strip().split(',')[0])
    
        #Designation
        Designation.append(Faculty.find('div', class_='gdlr-core-personnel-list-position').text.strip())
        
        #Email
        Email.append(Faculty.find('div', class_='kingster-personnel-info-list kingster-type-email').text.strip())
        
        #Department
        Department.append(soup2.find('h1', class_='kingster-page-title').text)
        
        #Extension
        tmp=Faculty.find('div', class_='kingster-type-phone').get_text(strip=True)
        if len(tmp)>1:
            Extension.append(int(tmp))
        else:
            Extension.append(int(-1))
        
        #Profile Link
        Profile_URL.append(Faculty.find('a')['href'])

for i in range(len(Profile_URL)):
    prof = Profile_URL[i]
    
    req3 = requests.get(prof)
    soup3 = BeautifulSoup(req3.content, 'html.parser')
    

    #HEC Status
    if 'HEC' in soup3.find('span',class_='gdlr-core-skin-caption').text.strip():
        HEC_Status.append("True")
    else:
        HEC_Status.append("False")
        
    #Profile Picture
    prof_link=soup3.find('a',class_='gdlr-core-lightgallery gdlr-core-js')
    if prof_link:
        ImageURL.append(prof_link['href'])
    else:
        ImageURL.append('NULL')
      
    #ID
    id_tag = soup3.find('script', id='burst-js-extra')
    pattern = r'"page_id":"(\d+)"'
    match = re.search(pattern, id_tag.string)
    if match:
        Id.append(int(match.group(1)))
    else:
        Id.append(-1)

    #Education
    booll=False
    all_div=soup3.find_all('div',class_="gdlr-core-pbf-element")
    for x in all_div:
        title=x.find_all('h3',class_="gdlr-core-title-item-title")
        for y in title:
            edu=y.text
            if edu == "Education":
                booll=True
                
                chk=x.find('span',"gdlr-core-title-item-caption")
                if chk:
                    Highest_Education.append(chk.get_text())
                
                next_div = x.find_next_sibling('div', class_="gdlr-core-pbf-element")
            
                if next_div:
                    txt = next_div.get_text()
                    Highest_Education.append(txt)
                
                else:
                    tmp=y.find('div',"gdlr-core-icon-list-content-wrap")
                    if tmp:
                        Highest_Education.append(tmp.get_text())
                    
            
    if booll == False:
        Highest_Education.append("No Education")
        
            
# Create a dictionary from the arrays
data = {
    'Name': Name,
    'Designation': Designation,
    'Email': Email,
    'Extension':Extension,
    'ImageURL':ImageURL,
    'ProfileURL':Profile_URL,
    'HECstatus':HEC_Status,
    'Highest_Education':Highest_Education,
    'Extension':Extension,
    'Department':Department
}

#Convert the dictionary to a DataFrame
df2= pd.DataFrame(data)

In [15]:
import time
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

file_path = 'ISB.html'

#Declaring Arrays for Storing Data
Id = []
Name = []
Designation = []
HEC_Status = []
Highest_Education = []
Email = []
Department = []
Extension = []
ImageURL = []
Profile_URL = []

with open(file_path, 'r', encoding='utf-8') as file:
    html_content = file.read()

soup1 = BeautifulSoup(html_content, 'html.parser')

#Dividing HTML in Containers
container = soup1.findAll('div', class_='container', id=True)

for cont in container:
    
    left = cont.find('div', class_='animated fadeInLeft').text.strip()
    right = cont.find('div', class_='animated fadeInRight').text.strip()
    Dep = (left + " " + right)
    
    #Finding Faculty Cards in Container
    fac_cards = cont.findAll('div', class_='col-lg-3')
    
    for i in range(len(fac_cards)):
        Faculty=fac_cards[i]
        
        #Name
        Name.append(Faculty.find('h3',class_='title').text.strip())
        
        #Designation
        Designation.append(Faculty.find('span',class_='post').text)
        
        #Department
        Department.append(Dep)
        
        #Email
        Email.append(Faculty.find('p').text)
        
        #Profile
        Profile_URL.append(Faculty.find('a')['href'])
        
        #ID
        url = Faculty.find('a')['href']
        Id.append(int(url[-4:]))

        
for i in range(len(Profile_URL)):
    prof = Profile_URL[i]
    
    req2 = requests.get(prof)
    soup2 = BeautifulSoup(req2.content, 'html.parser')
    
    #Image URL
    img = soup2.find('div',class_='img-thumbnail')
    ImageURL.append("http://isb.nu.edu.pk" + img.find('img')['src'])

    #HEC Approved
    hec=soup2.find('label',style='font-size:9pt;')
    if hec:
        HEC_Status.append(True)
    else:
        HEC_Status.append(False)
        
    #Extension
    check1=soup2.find('div',class_='profile-info')
    check2=check1.find_all('p',style="line-height: 20px;margin-bottom: 0px;")
    check2=check2[3]
    check3=check2.find('i',class_="fa fa-phone")
    if check3:
        span=check2.find('span')
        text = span.get_text(strip=True)
        pattern = r'Ext:\s*(\d{3})'
        match = re.search(pattern, text)
        if match:
            Extension.append(int(match.group(1)))
    else:
        Extension.append(int(-1))
        

In [17]:
for i in range(len(Profile_URL)):
    prof = Profile_URL[i]
    
    #Education
    options = Options()
    options.add_argument('--headless')
    
    dri = webdriver.Chrome(options=options)
    dri.get(prof)
    time.sleep(1)
    html = dri.page_source
    dri.quit()

    soup = BeautifulSoup(html, 'html.parser')
    edu = soup.find_all('span', id="DegreeName")  
    if edu:
        temp=edu[0].find('label')
        if temp:
            Highest_Education.append(temp.get_text())
        else:
            Highest_Education.append("No Education Record")
    else:
        Highest_Education.append("No Education Record")

        
data = {
    'Name': Name,
    'Designation': Designation,
    'Email': Email,
    'Extension':Extension,
    'ImageURL':ImageURL,
    'ProfileURL':Profile_URL,
    'HECstatus':HEC_Status,
    'Highest_Education':Highest_Education,
    'Extension':Extension,
    'Department':Department
}
    
#Convert the dictionary to a DataFrame
df3 = pd.DataFrame(data)

In [33]:
#Step 1 Conversion To Indvidual Faculty Files
df1.to_csv('lhr.csv', index=False)
df2.to_csv('khi.csv', index=False)
df3.to_csv('isb.csv', index=False)

#Step 2 Concatenation Of Files
resultant = pd.concat([df1, df2 , df3], axis=0)
resultant.to_csv('fast_faculty.csv',index=False)

#Step 3 Creating Chunk from Concatenated File
faculty = pd.read_csv("fast_faculty.csv")
roll_num = 2072
last_digit = roll_num % 10

if last_digit == 0:
    val = 0.1
else:
    val = last_digit / 10.0

_sample = faculty.sample(frac=val)
_sample.to_csv('sample.csv', index=False)